# Graph Construction
In this notebook we will construct the graph a bit differently from how we were doing it before to be more condusive to doing evaluation.  The approach will be as follows.
 - remove nodes with label of -1 --> was making data highly-balanced, skewing results
 - select a subset of nodes to do evaluation --> do not include these in any graph
 - build 2 graphs --> one connected based on ground truth (supervised), other on Jaccard (unsupervised)
 - evaluate periodically during training process as follows:
     - one node at a time
     - from holdout nodes, connect node to its original neighbors in the unsupervised
     - pass in this node's two-hop subgraph through our network
     - evaluate
     

In [1]:
import numpy as np
import pandas as pd
import torch as th
import dgl
import scipy
import networkx as nx
from progressbar import ProgressBar

Using backend: pytorch


## Load in Data

In [2]:
#load in our data - ~1mil rows per dataset
sup_neighbors = np.load('../hotel50k_graph_data/neighbors_sup.npy')
unsup_neighbors = np.load('../hotel50k_graph_data/neighbors_unsup.npy')
node_names = np.load('../hotel50k_graph_data/train_images_name.npy')
node_features = np.load('../hotel50k_graph_data/db_vectors.npy')

## Preprocess Data

In [3]:
#find which nodes have label -1 --> remove these
temp_node_names = node_names.tolist()
temp_node_names = [int(s.split('/')[0]) for s in temp_node_names]

#find all indeces (nodes) where node label is -1
minus_one_indeces = [i for i,x in enumerate(temp_node_names) if x == -1]

In [4]:
#in our neighbors, make sure there are no nodes that have label -1 --> idenitfy all of these nodes, we will remove them later
remap_dict = {x: -1 for x in minus_one_indeces}
def mp(entry):
    #go through our matrices, return -1 if node needs to be replaced or else just return the node
    return remap_dict[entry] if entry in remap_dict else entry
mp = np.vectorize(mp)

#update our neighbor matrices so now we know which neighbors to ignore (ie. label -1)
unsup_neighbors = mp(unsup_neighbors)
sup_neighbors = mp(sup_neighbors)

## Preprocess Graph Structure

In [5]:
#initialize the number of nodes and neighbors as two variables for ease of reference
#these dimensions are the same for the sup and unsup matrices
n = unsup_neighbors.shape[0]
m = unsup_neighbors.shape[1]
index = list(range(n))

### Unsupervised Graph Processing

In [6]:
#use pandas to create an adjacency list as a dict of lists --> {0:[76,23,90], ...}
unsup_df = pd.DataFrame(data = unsup_neighbors, index=index)

#create adjacency list
unsup_adj_list = unsup_df.T.to_dict('list')

In [7]:
#remove all nodes that have label -1
for k in minus_one_indeces:
    del unsup_adj_list[k]

In [8]:
#remove all possible neighbors that have label -1
for k,v in unsup_adj_list.items():
    v = [x for x in v if x!=-1]
    unsup_adj_list[k] = v

In [9]:
#extract validation nodes
from random import sample

possible_val_keys = list(unsup_adj_list.keys())
val_nodes = sample(possible_val_keys, 40000) #sample 40000 nodes for validation set ~10%

#make sure no training nodes have neighbors from validation set
#also make sure no training nodes have neighbors from validation set (important when we put them back in for evaluation)
temp_val_nodes = set(val_nodes)
for k,v in unsup_adj_list.items():
    v = [x for x in v if x not in temp_val_nodes] #make sure no node can have neighbors from validation set
    v = v[0:25]
    unsup_adj_list[k] = v

In [10]:
#extract our validation nodes and their neighbors to which we will reconnect
validation_nodes_and_neighbors = {k: unsup_adj_list[k] for k in val_nodes}

#now remove the validation nodes from the unsupervised adjacency list
for k in val_nodes:
    del unsup_adj_list[k]

### Supervised Graph Processing

In [11]:
sup_df = pd.DataFrame(data = sup_neighbors, index=index)

#create adjacency list
sup_adj_list = sup_df.T.to_dict('list')

In [12]:
#remove all nodes that have label -1
for k in minus_one_indeces:
    del sup_adj_list[k]

In [13]:
#remove all possible neighbors that have label -1
for k,v in sup_adj_list.items():
    v = [x for x in v if x!=-1]
    sup_adj_list[k] = v

In [14]:
#make sure no training nodes have neighbors from validation set
#also make sure no training nodes have neighbors from validation set (important when we put them back in for evaluation)
temp_val_nodes = set(val_nodes) #easier to search through --> O(1) time
for k,v in sup_adj_list.items():
    v = [x for x in v if x not in temp_val_nodes] #make sure no node can have neighbors from validation set
    v = v[0:25]
    sup_adj_list[k] = v

In [15]:
#now remove the validation nodes from the supervised adjacency list
for k in val_nodes:
    del sup_adj_list[k]

### Get Labels and Features based on Graphs
Now that we've dropped all nodes with label -1 and have extracted nodes (and stored their neighbors) that will be used for validation, we need to properly alter our label vector and features matrix to reflect which nodes we've selected for training.  We also need to make sure we store the validation nodes, neighbors, and features because we'll need that later

In [16]:
#fix weird case where there is no label 8 --> map label 92 to 8
node_labels = np.array(temp_node_names)
node_labels[node_labels==92] = 8

In [17]:
#create new nested dict with following structure --> {node1: {neighbors:[], features:[], label:_},  node2:...}
validation_dict = {}
for k,v in validation_nodes_and_neighbors.items():
    neighbors = v #store the neighbors of our node, k
    label = node_labels[k] #get label
    features = node_features[k] #get feature vector
    
    assert label != -1 #make sure we didn't mess up and have any label of -1
    
    #we will use/populate this dict as the value for each node in our validation set
    temp_dict = {'neighbors':neighbors, 'features':features, 'label':label}
    
    #create entry in our validation dictionary
    validation_dict[k] = temp_dict

In [18]:
#save our validation info as pickle file
import pickle
with open('validation_data.pickle', 'wb') as handle:
    pickle.dump(validation_dict, handle)

In [19]:
#drop all rows (corresponds to nodes) of features and labels that correspond to labels of -1 or validation nodes
#this way we'll be left with our train features and train labels
indeces_to_drop = [*list(validation_dict.keys()), *list(minus_one_indeces)]
train_features = np.delete(node_features,indeces_to_drop, axis = 0)
train_labels = np.delete(node_labels,indeces_to_drop, axis = 0)

### Checks for Correctness
Now that we've created the necessary data structures to create our training graphs and validation data, let's heck to make sure everything is correct 

In [20]:
#do we have the right number of nodes for each structure
assert train_labels.shape[0] == train_features.shape[0] == len(sup_adj_list) == len(unsup_adj_list)

#do we have the same nodes captured in the supervised and unsupervised graphs
assert set(sup_adj_list.keys()) == set(set(unsup_adj_list.keys()))

#make sure that there are no validation nodes in the training data
assert len((set(sup_adj_list.keys()) | set(unsup_adj_list.keys())) & set(validation_dict.keys())) == 0

### Dgl Remapping Problem
DGL will atuomatically remap nodes to consecutive integers.  This is a problem because in our validation set, the nodes and their neighbors are mapped according to the original order.
Solution
- create networkx graph
- create mapping dictionary of our current nodes to consecutive integers
- save this mapping
- apply this mapping to our validation nodes and their neighbors when we need to

In [21]:
#remap unsupervised graph --> same should work for supervised graph
unsup_g = nx.from_dict_of_lists(unsup_adj_list, nx.DiGraph)
sup_g = nx.from_dict_of_lists(sup_adj_list, nx.DiGraph)

In [22]:
#get an ordered list of all of our nodes
ordered_nodes = list(sorted(unsup_g.nodes))

assert ordered_nodes == list(sorted(sup_g.nodes)) #quick check sup/unsup have same nodes
assert list(unsup_g.nodes) == list(sorted(unsup_g.nodes)) #quick check that nodes were already in increasing order

#create dict to map all nodes in our graphs to consecutive integers
#we will need this same mapping dict to do the same for the 
remapping_dict = dict(zip(ordered_nodes, range(len(ordered_nodes))))

#relabel graphs
relabeled_unsup_g = nx.relabel_nodes(unsup_g, mapping = remapping_dict)
relabeled_sup_g = nx.relabel_nodes(sup_g, mapping = remapping_dict)

#### Fix Validation Remapping

In [23]:
with open('validation_data.pickle', 'rb') as f:
    validation_dict = pickle.load(f)

In [24]:
#make sure that the nodes we just remapped to matches what we do for validation
remapped_validation_dict = {}
i = max(relabeled_unsup_g) + 1 #counter for nodes

for k,v in validation_dict.items():
    k_new = i #indicate validation node
    
    v_new = v.copy()
    
    #remap neighbors
    v_new['neighbors'] = [remapping_dict.get(item,item) for item in v_new['neighbors']]
    
    #populate our new validation dict
    remapped_validation_dict[k_new] = v_new
    
    i+=1

In [25]:
#save validation info
with open('final_validation_data.pickle', 'wb') as f:
    pickle.dump(remapped_validation_dict, f)

## Create Training Graphs
We have done our checks and have our data.  Now let's make our two graphs --> supervised and unsupervised.  We will also include features and labels for each node.

In [26]:
#create dgl graph objects from networkx graphs
unsup_dgl_graph = dgl.from_networkx(relabeled_unsup_g)
sup_dgl_graph = dgl.from_networkx(relabeled_sup_g)

### Pass in Features, Labels

In [36]:
#pass in features to graph as a tensor
#features extracted via MobileNet
unsup_dgl_graph.ndata['features'] = th.tensor(train_features)
sup_dgl_graph.ndata['features'] = th.tensor(train_features)

#pass in labels
#pass in labels to our graphs
unsup_dgl_graph.ndata['labels'] = th.tensor(train_labels)
sup_dgl_graph.ndata['labels'] = th.tensor(train_labels)

### Save Graphs

In [39]:
#save each graph object
from dgl.data.utils import save_graphs
graph_labels = {"glabel": th.tensor([0,1])}
save_graphs("new_train_graphs.bin", [unsup_dgl_graph, sup_dgl_graph], graph_labels)